In [1]:
import tensorflow

In [2]:
# USAGE
# python train.py

# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")

# import the necessary packages
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from sklearn.metrics import classification_report
#from pyimagesearch import config
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import pickle
import os

Using TensorFlow backend.


In [3]:
def plot_training(H, N, plotPath):
	# construct a plot that plots and saves the training history
	plt.style.use("ggplot")
	plt.figure()
	plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
	plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
	plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
	plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
	plt.title("Training Loss and Accuracy")
	plt.xlabel("Epoch #")
	plt.ylabel("Loss/Accuracy")
	plt.legend(loc="lower left")
	plt.savefig(plotPath)


In [4]:
BATCH_SIZE = 32
# initialize the list of class label names
CLASSES = ["Employees","Normal_Customer","VIP_Customer"]
BASE_PATH = r'C:\Users\AS5271\Desktop\fine-tuning-keras\dataset'
# set the path to the serialized model after training
MODEL_PATH = r'C:\Users\AS5271\Desktop\fine-tuning-keras\output\VGG16-first.model'
 
# define the path to the output training history plots
UNFROZEN_PLOT_PATH = r'C:\Users\AS5271\Desktop\fine-tuning-keras\output\unfrozen_VGG16-first.png'
WARMUP_PLOT_PATH = r'C:\Users\AS5271\Desktop\fine-tuning-keras\output\warmup_VGG16-first.png'

In [5]:
# derive the paths to the training, validation, and testing
# directories
trainPath =r'C:\Users\AS5271\Desktop\fine-tuning-keras\dataset\training' 
valPath =r'C:\Users\AS5271\Desktop\fine-tuning-keras\dataset\vAL'
testPath =r'C:\Users\AS5271\Desktop\fine-tuning-keras\dataset\evaluation'

# determine the total number of image paths in training, validation,
# and testing directories
totalTrain = len(list(paths.list_images(trainPath)))
print(totalTrain)
totalVal = len(list(paths.list_images(valPath)))
print(totalVal)
totalTest = len(list(paths.list_images(testPath)))
print(totalTest)

360
120
120


In [6]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
	rotation_range=30,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

# initialize the validation/testing data augmentation object (which
# we'll be adding mean subtraction to)
valAug = ImageDataGenerator()

# define the ImageNet mean subtraction (in RGB order) and set the
# the mean subtraction value for each of the data augmentation
# objects
mean = np.array([123.68, 116.779, 103.939], dtype="float32")
trainAug.mean = mean
valAug.mean = mean

In [7]:
# initialize the training generator
trainGen = trainAug.flow_from_directory(
	trainPath,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=True,
	batch_size=BATCH_SIZE)

# initialize the validation generator
valGen = valAug.flow_from_directory(
	valPath,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=BATCH_SIZE)

# initialize the testing generator
testGen = valAug.flow_from_directory(
	testPath,
	class_mode="categorical",
	target_size=(224, 224),
	color_mode="rgb",
	shuffle=False,
	batch_size=BATCH_SIZE)

Found 360 images belonging to 3 classes.
Found 120 images belonging to 3 classes.
Found 120 images belonging to 3 classes.


In [8]:
# load the VGG16 network, ensuring the head FC layer sets are left
# off
baseModel = VGG16(weights="imagenet", include_top=False,
	input_tensor=Input(shape=(224, 224, 3)))

# construct the head of the model that will be placed on top of the
# the base model
headModel = baseModel.output
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(512, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(CLASSES), activation="softmax")(headModel)


W0717 09:46:34.751136 18652 deprecation_wrapper.py:119] From C:\Users\AS5271\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0717 09:46:34.766764 18652 deprecation_wrapper.py:119] From C:\Users\AS5271\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0717 09:46:34.766764 18652 deprecation_wrapper.py:119] From C:\Users\AS5271\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0717 09:46:34.784105 18652 deprecation_wrapper.py:119] From C:\Users\AS5271\AppData\Roaming\Python\Python37\site-packages\keras\backend\tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W07

In [9]:
# place the head FC model on top of the base model (this will become
# the actual model we will train)
model = Model(inputs=baseModel.input, outputs=headModel)

# loop over all layers in the base model and freeze them so they will
# *not* be updated during the first training process
for layer in baseModel.layers:
	layer.trainable = False

In [10]:
# compile our model (this needs to be done after our setting our
# layers to being non-trainable
print("[INFO] compiling model...")
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the head of the network for a few epochs (all other layers
# are frozen) -- this will allow the new FC layers to start to become
# initialized with actual "learned" values versus pure random
print("[INFO] training head...")
H = model.fit_generator(
	trainGen,
	steps_per_epoch=75,
	validation_data=valGen,
	validation_steps=10,
	epochs=50)

# reset the testing generator and evaluate the network after
# fine-tuning just the network head
print("[INFO] evaluating after fine-tuning network head...")
testGen.reset()
predIdxs = model.predict_generator(testGen,
	steps=(totalTest // BATCH_SIZE) + 1)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testGen.classes, predIdxs,
	target_names=testGen.class_indices.keys()))
plot_training(H, 50, WARMUP_PLOT_PATH)

# reset our data generators
trainGen.reset()
valGen.reset()

# now that the head FC layers have been trained/initialized, lets
# unfreeze the final set of CONV layers and make them trainable
for layer in baseModel.layers[15:]:
	layer.trainable = True

# loop over the layers in the model and show which ones are trainable
# or not
for layer in baseModel.layers:
	print("{}: {}".format(layer, layer.trainable))

# for the changes to the model to take affect we need to recompile
# the model, this time using SGD with a *very* small learning rate
print("[INFO] re-compiling model...")
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])

# train the model again, this time fine-tuning *both* the final set
# of CONV layers along with our set of FC layers
H = model.fit_generator(
	trainGen,
	steps_per_epoch=totalTrain // BATCH_SIZE,
	validation_data=valGen,
	validation_steps=totalVal // BATCH_SIZE,
	epochs=20)

# reset the testing generator and then use our trained model to
# make predictions on the data
print("[INFO] evaluating after fine-tuning network...")
testGen.reset()
predIdxs = model.predict_generator(testGen,
	steps=(totalTest // BATCH_SIZE) + 1)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testGen.classes, predIdxs,
	target_names=testGen.class_indices.keys()))
plot_training(H, 20, UNFROZEN_PLOT_PATH)

# serialize the model to disk
print("[INFO] serializing network...")
model.save(MODEL_PATH)

W0717 09:46:39.267457 18652 deprecation_wrapper.py:119] From C:\Users\AS5271\AppData\Roaming\Python\Python37\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0717 09:46:39.343891 18652 deprecation.py:323] From C:\Users\AS5271\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


[INFO] compiling model...
[INFO] training head...
Epoch 1/50
75/75 [==============================] - 463s 6s/step - loss: 7.1373 - acc: 0.4858 - val_loss: 6.7863 - val_acc: 0.5526
Epoch 2/50
75/75 [==============================] - 1179s 16s/step - loss: 6.2342 - acc: 0.5687 - val_loss: 7.1492 - val_acc: 0.5169
Epoch 3/50
75/75 [==============================] - 1188s 16s/step - loss: 5.6604 - acc: 0.6142 - val_loss: 5.0979 - val_acc: 0.6283
Epoch 4/50
75/75 [==============================] - 1183s 16s/step - loss: 5.1488 - acc: 0.6442 - val_loss: 6.0876 - val_acc: 0.6081
Epoch 5/50
75/75 [==============================] - 1183s 16s/step - loss: 4.8943 - acc: 0.6624 - val_loss: 5.4647 - val_acc: 0.6316
Epoch 6/50
75/75 [==============================] - 1177s 16s/step - loss: 4.3559 - acc: 0.6979 - val_loss: 6.5756 - val_acc: 0.5642
Epoch 7/50
75/75 [==============================] - 1191s 16s/step - loss: 4.4450 - acc: 0.6968 - val_loss: 6.7202 - val_acc: 0.5428
Epoch 8/50
75/75 [===

Epoch 1/20
11/11 [==============================] - 136s 12s/step - loss: 0.8041 - acc: 0.9290 - val_loss: 5.1340 - val_acc: 0.6042
Epoch 2/20
11/11 [==============================] - 121s 11s/step - loss: 1.0985 - acc: 0.9028 - val_loss: 4.6316 - val_acc: 0.6364
Epoch 3/20
11/11 [==============================] - 121s 11s/step - loss: 2.2309 - acc: 0.8329 - val_loss: 4.7206 - val_acc: 0.6136
Epoch 4/20
11/11 [==============================] - 123s 11s/step - loss: 1.8255 - acc: 0.8464 - val_loss: 8.4240 - val_acc: 0.3977
Epoch 5/20
11/11 [==============================] - 126s 11s/step - loss: 2.3737 - acc: 0.7843 - val_loss: 5.8055 - val_acc: 0.5625
Epoch 6/20
11/11 [==============================] - 124s 11s/step - loss: 1.6316 - acc: 0.8500 - val_loss: 4.1446 - val_acc: 0.6250
Epoch 7/20
11/11 [==============================] - 124s 11s/step - loss: 2.1339 - acc: 0.8178 - val_loss: 3.7667 - val_acc: 0.6818
Epoch 8/20
11/11 [==============================] - 121s 11s/step - loss: 1.